# As a note, many different imports are used for running the various results and models. ClarkeErrorGrid is a separate .py file that is imported in order to produce the clarke error grid graphs. ClarkeErrorGrid.py can be found in the same github repository as this file

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
from numpy import array
from numpy import hstack
import numpy.linalg as la
import pandas as pd
from matplotlib import pyplot as plt

import xml.etree.ElementTree as et 

from datetime import datetime
from datetime import timedelta

import glob
import os

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import r2_score

from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from math import sqrt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic as order_selecte
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR

from filterpy.kalman import KalmanFilter
from filterpy.kalman import UnscentedKalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints

from scipy.stats.distributions import norm
from scipy.optimize import fmin

from pydataset import data as pydata
from collections import defaultdict

import time
import tensorflow as tf
from ClarkeErrorGrid import clarke_error_grid

import pickle
from tqdm import tqdm
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed    

from fbprophet import Prophet

In [ ]:
metric_labels = ["RMSE","MAE","TP_hypo","FP_hypo","TN_hypo","FN_hypo","TP_hyper","FP_hyper","TN_hyper","FN_hypter","MC_hypo","MC_hyper","R2","Zone A", "Zone B", "Zone C", "Zone D", "Zone E", "Perc. Safe", "Perc. Unnecessary", "Perc. Dangerous"]
def get_metrics(actual,predicted):
    """
    Get metrics return as tuple regarding the various metrics we wish to use when analyzing how well an algorithm is predicting
    
    Parameters:
        actual(ndarray, (n,)): The actual blood glucose levels for a given time series
        predicted (ndarray, (n)): The predicted blood glucose levels for a given time series
    
    Returns:
        RMSE: The Root Mean Square Error of the prediction
        MAE: The Mean Absolute Error of the prediction
        TP_hypo: The true positive rate for predicted Hypoglycaemic (<70) events
        FP_hypo: The false positive rate for predicted Hypoglycaemic (<70) events
        TN_hypo: The true negative rate for predicted hypoglycaemic (< 70) events
        FN_hypo: The false negative rate for predicted hypoglycaemic (< 70) events
        TP_hyper: The true positive rate for predicted Hyperglycaemic events (>180)
        FP_hyper: The false positive rate for predicted hyperglycaemic events (>180)
        TN_hyper: The true negative rate for predicted hyperglycaemic events(> 180)
        FN_hypter: The false negative rate for predicted hyperglycaemic events (>180)
        MC_hypo: The Matthew Correlation Coefficient predicted for hypoglycaemic (<70) events
        MC_hyper: The Matthew Correlation Coefficient predicted for hyperglycaemic (> 180) events
        r2: The R^2 score for the given predictions
    """
    RMSE = mean_squared_error(actual,predicted,squared=False)
    MAE = mean_absolute_error(actual,predicted)
    actual_hypo = np.where(actual < 70,1,0)
    actual_hyper = np.where(actual > 180,1,0)
    predicted_hypo = np.where(predicted < 70,1,0)
    predicted_hyper = np.where(predicted > 180,1,0)
    tn_hypo,fp_hypo,fn_hypo,tp_hypo = confusion_matrix(actual_hypo,predicted_hypo).ravel()
    mc_hypo = matthews_corrcoef(actual_hypo,predicted_hypo)
    tn_hyper,fp_hyper,fn_hyper,tp_hyper = confusion_matrix(actual_hyper,predicted_hyper).ravel()
    mc_hyper = matthews_corrcoef(actual_hyper,predicted_hyper)
    r2 = r2_score(actual,predicted)
    return RMSE, MAE, tp_hypo,fp_hypo,tn_hypo,fn_hypo,tp_hyper,fp_hyper,tn_hyper,fn_hyper,mc_hypo,mc_hyper,r2

## This code makes the assumption that you already have access to the OhioT1DM dataset, and that the corresponding data (as of 2020) is found in the folder ./OHIODATA. 

In [ ]:
def parse_XML(xml_file): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    #use the columns as defined in the OhioT1DM Dataset 2020
    df_cols = ["date","glucose_level", "finger_stick", "basal", "temp_basal","bolus","meal","sleep",
           "work","stressors","hypo_event","illness","exercise","basis_heart_rate","basis_gsr",
          "basis_skin_temperature","basis_air_temperature","basis_steps","basis_sleep","acceleration"]
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    #Goes row by row from the XML file and appends to an array which will be converted to a PANDAS dataframe
    for child in xroot:
        if child.tag == df_cols[1]:
            final = []
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[1] = float(val.get("value"))
                final.append(res)
            
        elif child.tag == df_cols[2]:
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[2] = float(val.get("value"))
                final.append(res)

        elif child.tag == df_cols[3]:
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[3] = float(val.get("value"))
                final.append(res)
                
        elif child.tag == df_cols[4]:
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts_begin"))
                res[4] = float(val.get("value"))
                final.append(res)

        elif child.tag == df_cols[5]:
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts_begin"))
                res[5] = float(val.get("dose"))
                final.append(res)
                
        elif child.tag == "meal":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[6] = float(val.get("carbs"))
                final.append(res)
                
        elif child.tag == "sleep":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("tbegin"))
                res[7] = np.nan  #FOR THIS FIND THE DIFFERENCE BETWEEN tbegin AND tend
                final.append(res)        
        elif child.tag == "work":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("tbegin"))
                res[8] = np.nan  #FOR THIS FIND THE DIFFERENCE BETWEEN tbegin AND tend
                final.append(res)        
        elif child.tag == "stressors":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[9] = 1
                final.append(res)        
        elif child.tag == "hypo_event":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[10] = np.nan 
                final.append(res)  
        elif child.tag == "illness":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts_begin"))
                res[11] = np.nan
                final.append(res)  
        elif child.tag == "exercise":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[12] = float(val.get("intensity")) * float(val.get("duration"))
                final.append(res)  
        elif child.tag == "basis_heart_rate":
            print("basis_heart_rate not included")
        elif child.tag == "basis_gsr":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[14] = float(val.get("value"))
                final.append(res)
        elif child.tag == "basis_skin_temperature":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[15] = float(val.get("value"))
                final.append(res)
        elif child.tag == "basis_air_temperature":
            print("basis_air_temperature not included")
        elif child.tag == "basis_steps":
            print("basis_steps not included")
        elif child.tag == "basis_sleep":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("tbegin"))
                res[18] = np.nan  #FOR THIS FIND THE DIFFERENCE BETWEEN tbegin AND tend
                final.append(res)
        elif child.tag == "acceleration":
            for val in child.getchildren():
                res = [np.nan] * 20
                res[0] = str(val.get("ts"))
                res[19] = float(val.get("value"))
                final.append(res)
      
    #CONVERT ARRAY TO PANDAS DATAFRAME
    df = pd.DataFrame(final, columns = df_cols)
    df[df['date'] == "None"] = np.nan
    df.dropna(subset=["date"],inplace=True)
    
    #convert date column into a datetime column, then remove date 
    df['datetime'] = pd.to_datetime(df['date'],format='%d-%m-%Y %H:%M:%S')
    df = df.set_index('datetime')
    df.drop(['date'], axis=1, inplace=True)
    
    
    #Since the array is mostly sparse and not aligned by time, merging the dataframes to align multiple data values allows for better multivariate analysis using time series
    #The first block initializes the new dataframe merging, the following loop goes through the rest of the columns
    #Each merge is done backward since we can't use data from the future to predict the future.
    #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html
    glu = df["glucose_level"].dropna()
    finger = df["finger_stick"].dropna()
    new_df = pd.merge_asof(glu, finger,
                          on='datetime',
                          direction = "backward",
                          tolerance=pd.Timedelta('4T'))
    df_cols = ["basal", "temp_basal","bolus","meal","sleep",
               "work","stressors","hypo_event","illness","exercise","basis_heart_rate","basis_gsr",
              "basis_skin_temperature","basis_air_temperature","basis_steps","basis_sleep","acceleration"]
    
    for col in df_cols:
        temp = df[col].dropna()
        new_df = pd.merge_asof(new_df, temp,
                          on='datetime',
                          direction = "backward",
                          tolerance=pd.Timedelta('4T'))
    new_df = new_df.set_index(['datetime'])
    
    practice = new_df    #need to check where datetime is missing values, then extrapolate
    practice['deltaT'] = practice.index.to_series().diff().dt.total_seconds().div(60, fill_value=0)
    i = 1
    filled_dates = []
    index = practice.index.values
    index = pd.to_datetime(index)

    #FILLS MISSING VALUES: uses a rolling average 
    while i < len(practice)-1:
        if practice["deltaT"].values[i] > 6:
            start = index[i-1]      
            end = index[i]          
            last_date = start

            j = 1
            while last_date < end:
                last_date = last_date + timedelta(minutes=5)

                pred = np.mean(practice["glucose_level"].values[i-2*j:i-1])   
                j+=1
                filled_dates.append([last_date,pred,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None])
        i += 1

    #REMAKES THE DATAFRAME WITH FILLED DATES
    temp_df = pd.DataFrame(filled_dates, columns = ["date","glucose_level", "finger_stick", "basal", "temp_basal","bolus","meal","sleep",
           "work","stressors","hypo_event","illness","exercise","basis_heart_rate","basis_gsr",
          "basis_skin_temperature","basis_air_temperature","basis_steps","basis_sleep","acceleration"])
    temp_df['datetime'] = pd.to_datetime(temp_df['date'],format='%d-%m-%Y %H:%M:%S')
    temp_df = temp_df.set_index('datetime')
    temp_df.drop(['date'], axis=1, inplace=True)

    #ALIGNS THE INDEX BY TIME
    final_df = pd.concat([practice,temp_df])
    new_df = final_df.sort_index()
    #remove nans, and fill remaining with 0
    new_df.dropna(subset = ["glucose_level"], inplace = True)
    new_df = new_df.fillna(0)

    return new_df

In [ ]:
def create_metrics_file(metric_path,metrics_dict):
    """
    Creates a metric file containing the results of the get_metrics function for each patient for a given model
    
    Parameters:
        metric_path(str):The file path for the metric file
        metrics_dict(dictionary):Dictionary containing the results of get_metrics for each patient
    """
    with open(metric_path,"w+") as outfile:
        for key in metrics_dict.keys():
            outfile.write("Patient " + str(key) + "\n")
            for met,val in zip(metric_labels,metrics_dict[key]):
                outfile.write(met + ": " + str(val) + "\n")
            outfile.write("\n")
        rmse = np.mean([val[0] for val in metrics_dict.values()])
        mae = np.mean([val[1] for val in metrics_dict.values()])
        mcc_l = np.mean([val[10] for val in metrics_dict.values()])
        mcc_h  = np.mean([val[11] for val in metrics_dict.values()])
        r2 = np.mean([val[12] for val in metrics_dict.values()])
        outfile.write("Average RMSE: " + str(rmse) + "\n")
        outfile.write("Average MAE: " + str(mae) + "\n")
        outfile.write("Average MCC_hypo: " + str(mcc_l) + "\n")
        outfile.write("Average MCC_hyper: " + str(mcc_h) + "\n")
        outfile.write("Average R^2: " + str(r2) + "\n")
    return

In [ ]:
def create_directories(root,leafs):
    """
    Creates all need directories for creating files.
    Parameters:
        root(str):the name of the root directory to create
        leafs(list):list containing the leaf nodes to create
    Returns:
        None
    Example:
    root='pickle'
    leafs=['linear','ridge','svm/rbf']
    
    This above example with root 'pickle' and the given leafs will create the following filepath and directories:
    pickle/30min/linear, pickle/30min/ridge, pickle/30min/svm/rbf, pickle/60min/linear, pickle/60min/ridge, pickle/60min/svm/rbf
    
    """
    base30 = root + "/30min/"
    base60 = root + "/60min/"
    for leaf in leafs:
        path30 = base30 + leaf
        path60 = base60 + leaf
        if not os.path.exists(path30):
            os.makedirs(path30)
        if not os.path.exists(path60):
            os.makedirs(path60)

In [ ]:
def create_pickle_dir():
    """
    Creates the pickle directory, along with all needed subdirectories
    """
    root = "pickle"
    leafs = ["linear","ridge","lasso","elastic","svm/poly","svm/sigmoid/","svm/rbf","knn","rf","gradient","xgb","mlp"]
    create_directories(root,leafs)
    return

In [ ]:
def create_plot_dir():
    """
    Creates plots directory, along with all needed subdirectories

    """
    root = "plots"
    leafs = ["linear","ridge","lasso","elastic","svm/poly","svm/sigmoid/","svm/rbf","knn","rf","gradient","xgb","mlp"]
    create_directories(root,leafs)
    return

In [ ]:
def create_clarke_dir():
    """
    Creates the clarke directory, along with all needed subdirectories
    """
    root = "clarke"
    leafs = ["linear","ridge","lasso","elastic","svm/poly","svm/sigmoid/","svm/rbf","knn","rf","gradient","xgb","mlp"]
    create_directories(root,leafs)
    return

In [ ]:
def create_metrics_dir():
    """
    Creates the metrics directory, along with all needed subdirectories
    """
    root = "metrics"
    leafs = ["linear","ridge","lasso","elastic","svm/poly","svm/sigmoid/","svm/rbf","knn","rf","gradient","xgb","mlp"]
    create_directories(root,leafs)
    return

In [ ]:
def create_all_directories():
    """
    Creates the pickle,plots,clarke, and metrics directories, along with all needed subdirectories

    """
    create_pickle_dir()
    create_plot_dir()
    create_clarke_dir()
    create_metrics_dir()
    return

In [ ]:
def comparison_plot(xvals,actual,pred,plot_title,filepath):
    """
    Creates a plot of the predicted blood glucose values against the actual blood glucose values. Saves the plot in a 
    figure
    Parameters:
        xvals(list): Range for plotting the glucose values. Since data is time indexed, this is just an list from 0 to the number of 
        blood glucose values
        actual((n,) ndarray): Array containing the actual blood glucose values
        pred((n,) ndarray): Array containing the predicted blood glucose values
        plot_title(str): String for the name of the plot
        filepath(str): File path for storing the plot
    """
    fig,ax = plt.subplots()
    ax.plot(xvals,actual,'ob',label="actual",ms=0.5)
    ax.plot(xvals,pred,'or',label="predicted",ms=0.5,linewidth=1)
    ax.set_xlabel("time step")
    ax.set_ylabel("blood glucose (mg/dl)")
    ax.legend()
    plt.savefig(filepath,bbox_inches="tight")
    plt.show()
    return

In [ ]:
def create_test_train_dictionary():
    """
    Creates a map that maps the patient number to the test and train data for that patient
    Returns:
        test_train_dict(dictionary): dictionary mapping patient number to the test and train dataframes
    """
    #as mentioned earlier, the code assumes that you already have the OhioT1DM dataset and that this data is in the folder OHIODATA
    #as well, this code is specifically written for the 2020 OhioT1DM dataset
    test_train_dict = {}
    base = "OHIODATA/"
    test="OhioT1DM-2-testing/"
    train = "OhioT1DM-2-training/"
    end_test = "-ws-testing.xml"
    end_train = "-ws-training.xml"
    file_nums = [540,544,552,567,584,596]
    for num in file_nums:
        file = base + test+str(num)+end_test
        test_data = parse_XML(file)
        train_data = parse_XML(base+train+str(num)+end_train)
        test_train_dict[num] = {"test":test_data,"train":train_data}
    return test_train_dict


In [ ]:
def transform_to_regression(num_lags,pred_length,df,features=None):
    """
    Takes the time series data and turns the data into a regression problem by converting the previous timesteps
    into features and making label the next pred_length value of blood glucose values
    Parameters:
        num_lags(int): How many time steps (in 5 minutes) should be used to create the features
        pred_length(int): How far into the future (in 5 minute steps) should the label be
        df(pandas dataframe): dataframe containing the patient's information
        features(list or None): Which columns from the dataframe should be included when creating the features, if
            None, then use all columns
    """
    #total number of samples is the lag length + prediction length +1 since 0 indexed
    pred_tot = len(df)-(num_lags+pred_length-1)
    #create the corresponding labels
    gluc_values = df["glucose_level"].values
    labels=np.zeros((pred_tot,pred_length))
    for i in range(pred_length):
        labels[:,i] = gluc_values[num_lags+i:num_lags+pred_tot+i]
    if features is None:
        #transform for all columns
        features=df.columns.values
    first=True
    #create the features
    for feature in features:
        if(feature == "deltaT"):
            continue
        feature_vals = df[feature].values
        matrix = np.zeros((pred_tot,num_lags))
        for i in range(num_lags):
            matrix[:,i] = feature_vals[i:-num_lags-pred_length+i+1]
        #ensure matrix is copy so that data isn't accidently altered
        if(first):
            data = matrix.copy()
            first=False
        else:
            data = np.hstack((data,matrix))
    return data,labels

In [ ]:
def create_test_train_regression_dictionary(data_dictionary,ph,features=None):
    """
    Creates a dictionary that maps patient number to both the train and test data, and train and test labels
    Parameters:
        data_dictionary(pandas dataframe): dictionary mapping patient id to both train and test data frames
        ph(int): Prediction Horizon (time increments of 5 of how far to predict into the future)
        features(list or None): columns of the pandas dataframe to use when creating the features. If None, use all columns
    """
    #we only use the past 30 minutes to predict the next 30 minutes and 60 minutes
    past_values = 6
    train_dictionary = {}
    for key in data_dictionary.keys():
        train_data,train_labels = transform_to_regression(past_values,ph,data_dictionary[key]["train"],features)
        test_data,test_labels = transform_to_regression(past_values,ph,data_dictionary[key]["test"],features)
        train_dictionary[key] = {"test_data":test_data,"test_labels":test_labels,"train_data":train_data,"train_labels":train_labels}
    return train_dictionary

In [ ]:
def load_data():
    """
    Creates the train_test dictionary for each person, converts dataframes into regression data and labels, then stores in dictionary. Do this for 30 min horizon and 60 min horizon
    Returns:
        regression_dict_30: Dictionary containing regression data and labels for each patient for 30 min horizon
        regression_dict_60: Dictionary containing regression data and labels for each patient for 60 min horizon
    """
    people_dict= create_test_train_dictionary()
    regression_dict_30 = create_test_train_regression_dictionary(people_dict,6)
    regression_dict_60 = create_test_train_regression_dictionary(people_dict,12)

    return regression_dict_30,regression_dict_60,people_dict


In [ ]:
def produce_results(models,data,ph,model_name):
    """Produces the dictionaries needed to create the results files"""
    results_dict= {}
    loop = tqdm(total=6,position=0)
    
    for key in models.keys():
        model = models[key]
        pred = model.predict(data[key]["test_data"])
        met = get_metrics(data[key]["test_labels"][:,ph-1],pred[:,ph-1])
        print(met)
        df = pd.DataFrame(people_dict[key]["test"]["glucose_level"].iloc[11:])
        print(len(df))
        print(len(pred))
        df["pred_glucose_level"] = pred[:,ph-1]
        print("df",df)
        results_dict[key] = [df,met[:2]]
    create_results_files(results_dict)

In [ ]:
def predict_all(models,data,model_name,plot_path,clarke_path,metric_path,ph):
    """
    Creates the predictions for each model
    Parameters:
        models(dictionary): dictionary containing the trained model for each person
        data(dictionary): Dictionary containing the regression data for each person
        model_name(string): name of model to include in file name
        plot_path(str): Path to the directory where the plot for blood glucose will be stored
        clarke_path(str): Path to the directory where the clarke error grid will be stored
        metric_path(str): Path to the directory where the metric text file will be stored
        ph(int):Prediction horizon being used (needs to be same as corresonding prediction horizon as the dictionary being used)
    
    Returns:
        rmse: dictionary mapping patient number to RMSE for the specific ph
        mae: dictionary mapping patient number to MAE for the specific ph
    """
    
    metrics_dict = {}
    #used to measure how long training and predicting will take for all 6 patients
    loop = tqdm(total = 6,position=0)

    #for each patient, make the predictions, then plot the results, save the results, and measure with the clarke error grid
    for key in models.keys():
        model = models[key]
        pred = model.predict(data[key]["test_data"])
        xvals = len(data[key]["test_labels"])
        xvals = np.arange(xvals)
        met = get_metrics(data[key]["test_labels"][:,ph-1],pred[:,ph-1])

        file_name = str(key) + model_name + str(ph*5) + ".pdf"
        clarke_name = str(key) + model_name + str(ph*5) + "clarke" + ".pdf"
        
        zones = clarke_error_grid(data[key]["test_labels"][:,ph-1],pred[:,ph-1],"Patient " + str(key) + " " + str(ph*5) + " Minute Prediction",clarke_path+clarke_name)
        zones = tuple(zones)
        met = met + zones
        tot = np.sum(zones)
        safe = (zones[0] + zones[1])/tot
        unn = (zones[2])/tot
        danger = (zones[3] + zones[4])/tot
        met = met + (safe,unn,danger)
        metrics_dict[key] = met
        plot_title = "Patient " + str(key) + " " + str(ph*5) + " Minute Prediction " + model_name.capitalize() + " Regression"
        comparison_plot(xvals,data[key]["test_labels"][:,ph-1],pred[:,ph-1],plot_title,plot_path+file_name)
        loop.set_description("Finished iter in predict all")
        loop.update(1)
    metric_name = model_name + str(ph*5) + "metrics.txt"
    create_metrics_file(metric_path + metric_name,metrics_dict)
    return

In [ ]:
def predict_all_single(models,data,model_name,plot_path,clarke_path,metric_path,ph):
    """
    Creates the predictions for each model. Similar to predict_all, but does not make the assumption that the predictions contain more than one value for each time step
    Parameters:
        models(dictionary): dictionary containing the trained model for each person
        data(dictionary): Dictionary containing the regression data for each person
        model_name(string): name of model to include in file name
        plot_path(str): Path to the directory where the plot for blood glucose will be stored
        clarke_path(str): Path to the directory where the clarke error grid will be stored
        metric_path(str): Path to the directory where the metric text file will be stored
        ph(int):Prediction horizon being used (needs to be same as corresonding prediction horizon as the dictionary being used)
    
    Returns:
        rmse: dictionary mapping patient number to RMSE for the specific ph
        mae: dictionary mapping patient number to MAE for the specific ph
    """
    metrics_dict = {}
    loop = tqdm(total = 6,position=0)

    for key in models.keys():
        model = models[key]
        pred = model.predict(data[key]["test_data"])
        xvals = len(data[key]["test_labels"])
        xvals = np.arange(xvals)
        met = get_metrics(data[key]["test_labels"][:,ph-1],pred)

        file_name = str(key) + model_name + str(ph*5) + ".pdf"
        clarke_name = str(key) + model_name + str(ph*5) + "clarke" + ".pdf"
        
        zones = clarke_error_grid(data[key]["test_labels"][:,ph-1],pred,"Patient " + str(key) + " " + str(ph*5) + " Minute Prediction",clarke_path+clarke_name)
        zones = tuple(zones)
        met = met + zones
        metrics_dict[key] = met
        comparison_plot(xvals,data[key]["test_labels"][:,ph-1],pred,str(key) + " " + model_name,plot_path+file_name)
        loop.set_description("Finished iter in predict all")
        loop.update(1)
    metric_name = model_name + str(ph*5) + "metrics.txt"
    create_metrics_file(metric_path + metric_name,metrics_dict)
    return

In [ ]:
def create_results_files(results_dict30):#,results_dict60):
    """
    Writes the needed files for the competition results.
    The files are:
        results.txt: This is a text file that contains the results for the RMSE and MAE of all the patients. This file will need to be converted to a pdf
        
    Parameters:
            results_dict30: Dictionary that maps each person id to an iterable (list or tuple) containing the following structures
                results[0]: Contains a pandas dataframe with 3 columns. First column is the time value, the second column is the actual blood glucose values, and the third column is the predicted blood glucose values for the 30 minute prediction horizon
                results[1]: Is an iterable (list or tuple) with the following values in order: RMSE,MAE
                
            results_dict60: Dictionary that maps each person id to an iterable (list or tuple) containing the following structures
                results[0]: Contains a pandas dataframe with 3 columns. First column is the time value, the second column is the actual blood glucose values, and the third column is the predicted blood glucose values for the 60 minute prediction horizon
                results[1]: Is an iterable (list or tuple) with the following values in order: RMSE,MAE
    """
    patients = [540,544,552,567,584,596]
    with open("results.txt","w+") as results:
        results.write("30 Minute Prediction Horizon" +"\n" + "\n")
        rmse = 0
        mae = 0
        for patient in patients:
            file_name = "lasso " + str(patient) + " " + "30.txt"
            results_dict30[patient][0].to_csv(file_name,sep=" ")
            results.write("Patient " + str(patient) + "\n")
            results.write("RMSE: " + str(results_dict30[patient][1][0]) + "\n")
            results.write("MAE: " + str(results_dict30[patient][1][1]) + "\n" + "\n")
            rmse += results_dict30[patient][1][0]
            mae += results_dict30[patient][1][1]
        rmse = rmse / 6
        mae = mae / 6
        results.write("Average RMSE: " + str(rmse) + "\n")
        results.write("Average MAE: " + str(mae) + "\n" + "\n")
        
        """for patient in patients:
            file_name = "lasso " + str(patient) + " " + "60.txt"    
            np.savetxt(file_name,results_dict60[patient][0].values,delimiter = " ")
            results.write("Patient " + str(patient) + "\n")
            results.write("RMSE: " + results_dict60[patient][1][0] + "\n")
            results.write("MAE: " + results_dict60[patient][1][1] + "\n" + "\n")
            rmse += results_dict60[patient][1][0]
            mae += results_dict60[patient][1][1]
        rmse = rmse / 6
        mae = mae / 6
        results.write("Average RMSE: " + rmse + "\n")
        results.write("Average MAE: " + mae + "\n" + "\n")"""
    return

In [ ]:
#create all needed directories
create_all_directories()

In [ ]:
#define the global dictionaries to be used in training
regression_dict_30,regression_dict_60,people_dict = load_data()

In [ ]:
def average_glucose():
    """
    Calculates the average blood glucose level of the test set for each patient
    """
    patients = [540,544,552, 567, 584, 596]
    for key in patients:
        data = people_dict[key]["test"]["glucose_level"].values
        print("Patient " + str(key),"Average glucose level",np.mean(data))

In [ ]:
average_glucose()

# Support Vector Machine Regression
## RBF Kernel

In [ ]:
def train_rbf_models(data,ph):
    """
    Train the Support Vector Machine with RBF Kernel for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/svm/rbf/"
    for key in data.keys():
        rbf = SVR()
        rbf = MultiOutputRegressor(rbf)
        rbf.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "rbf" + str(ph*5)+ ".pickle"
        pickle.dump(rbf,open(path+name,'wb'))
    return

In [ ]:
def predict_rbf_models(data,ph):
    """
    Load each trained Support Vector Machine with RBF Kernel from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    models = {}
    path = "pickle/" + str(ph*5) + "min/svm/rbf/"
    file_path = "plots/" + str(ph*5) + "min/svm/rbf/"
    clarke_path = "clarke/" + str(ph*5) + "min/svm/rbf/"
    metric_path =  "metrics/" + str(ph*5) + "min/svm/rbf/"    
    for key in data.keys():
        name = str(key) + "rbf" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"rbf",file_path,clarke_path,metric_path,ph)

In [ ]:
train_rbf_models(regression_dict_30,6)
train_rbf_models(regression_dict_60,12)

In [ ]:
predict_rbf_models(regression_dict_30,6)
predict_rbf_models(regression_dict_60,12)

In [ ]:
predict_hard_filter_rbf_models(regression_dict_30,6,4)
predict_hard_filter_rbf_models(regression_dict_60,12,4)

## Polynomial Kernel

In [ ]:
def train_poly_models(data,ph):
    """
    Train the Support Vector Machine with Polynomial Kernel for each person and saves them to pickle files
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    loop = tqdm(total = 6,position=0)

    path = "pickle/" + str(ph*5) + "min/svm/poly/"
    for key in data.keys():
        poly = SVR(kernel="poly")
        poly = MultiOutputRegressor(poly)
        poly.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "poly" + str(ph*5)+ ".pickle"
        pickle.dump(poly,open(path+name,'wb'))
        loop.set_description("Time has finished")
        loop.update(1)
    return

In [ ]:
def predict_poly_models(data,ph):
    """
    Load each trained Support Vector Machine with Pollynomial Kernel from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/svm/poly/"
    file_path = "plots/" + str(ph*5) + "min/svm/poly/"
    clarke_path = "clarke/" + str(ph*5) + "min/svm/poly/"
    metric_path =  "metrics/" + str(ph*5) + "min/svm/poly/"    
    models = {}
    for key in data.keys():
        name = str(key) + "poly" + str(ph*5) + ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"poly",file_path,clarke_path,metric_path,ph)

In [ ]:
train_poly_models(regression_dict_30,6)
train_poly_models(regression_dict_60,12)

In [ ]:
predict_poly_models(regression_dict_30,6)
predict_poly_models(regression_dict_60,12)

## Sigmoid Kernel

In [ ]:
def train_sigmoid_models(data,ph):
    """
    Train the Support Vector Machine with Sigmoid Kernel for each person and saves them to pickle files
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    loop = tqdm(total = 6,position=0)

    path = "pickle/" + str(ph*5) + "min/svm/sigmoid/"
    for key in data.keys():
        sig = SVR(kernel="sigmoid")
        sig = MultiOutputRegressor(sig)
        sig.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "sig" + str(ph*5)+ ".pickle"
        pickle.dump(sig,open(path+name,'wb'))
        loop.set_description("Time has finished")
    return

In [ ]:
def predict_sigmoid_models(data,ph):
    """
    Load each trained Support Vector Machine with Sigmoid Kernel from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    models = {}
    path = "pickle/" + str(ph*5) + "min/svm/sigmoid/"
    file_path = "plots/" + str(ph*5) + "min/svm/sigmoid/"
    clarke_path = "clarke/" + str(ph*5) + "min/svm/sigmoid/"
    metric_path =  "metrics/" + str(ph*5) + "min/svm/sigmoid/"    
    for key in data.keys():
        name = str(key) + "sig" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"sigmoid",file_path,clarke_path,metric_path,ph)

In [ ]:
train_sigmoid_models(regression_dict_30,6)
train_sigmoid_models(regression_dict_60,12)

In [ ]:
predict_sigmoid_models(regression_dict_30,6)
predict_sigmoid_models(regression_dict_60,12)

# Linear Regression

In [ ]:
def train_linear_models(data,ph):
    """
    Train the Linear Regression Model with no regularization for each person and saves them to pickle files
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/linear/"
    for key in data.keys():
        lr = linear_model.LinearRegression(n_jobs=-1)
        lr = MultiOutputRegressor(lr)
        lr.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "lr" + str(ph*5)+ ".pickle"
        pickle.dump(lr,open(path+name,'wb'))
    return

In [ ]:
def predict_linear_models(data,ph):
    """
    Load each trained Linear Regression model with no regularization from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    models = {}
    path = "pickle/" + str(ph*5) + "min/linear/"
    file_path = "plots/" + str(ph*5) + "min/linear/"
    clarke_path = "clarke/" + str(ph*5) + "min/linear/"
    metric_path =  "metrics/" + str(ph*5) + "min/linear/"
    for key in data.keys():
        name = str(key) + "lr" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"lr",file_path,clarke_path,metric_path,ph)

In [ ]:
train_linear_models(regression_dict_30,6)
train_linear_models(regression_dict_60,12)

In [ ]:
predict_linear_models(regression_dict_30,6)
predict_linear_models(regression_dict_60,12)

# Ridge Regression

In [ ]:
def train_ridge_models(data,ph):
    """
    Train the Linear Regression Model with Ridge Regularization for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/ridge/"
    for key in data.keys():
        lr = linear_model.Ridge()
        lr = MultiOutputRegressor(lr)
        lr.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "ridge" + str(ph*5)+ ".pickle"
        pickle.dump(lr,open(path+name,'wb'))
    return

In [ ]:
def predict_ridge_models(data,ph):
    """
    Load each trained Linear Regression Model with Ridge Regularization from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    models = {}
    path = "pickle/" + str(ph*5) + "min/ridge/"
    file_path = "plots/" + str(ph*5) + "min/ridge/"
    clarke_path = "clarke/" + str(ph*5) + "min/ridge/"
    metric_path =  "metrics/" + str(ph*5) + "min/ridge/"
    for key in data.keys():
        name = str(key) + "ridge" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"ridge",file_path,clarke_path,metric_path,ph)

In [ ]:
train_ridge_models(regression_dict_30,6)
train_ridge_models(regression_dict_60,12)

In [ ]:
predict_ridge_models(regression_dict_30,6)
predict_ridge_models(regression_dict_60,12)

# Lasso Regression

In [ ]:
def train_lasso_models(data,ph):
    """
    Train the Linear Regression Model with Lasso Regularization for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/lasso/"
    loop = tqdm(total = 6,position=0)

    for key in data.keys():
        lr = linear_model.Lasso(max_iter=10000)
        lr = MultiOutputRegressor(lr)
        lr.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "lasso" + str(ph*5)+ ".pickle"
        pickle.dump(lr,open(path+name,'wb'))
        loop.set_description("Done")
        loop.update(1)
    return

In [ ]:
def predict_lasso_models(data,ph):
    """
    Load each trained Linear Model with Lasso Regularization from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/lasso/"
    file_path = "plots/" + str(ph*5) + "min/lasso/"
    clarke_path = "clarke/" + str(ph*5) + "min/lasso/"
    metric_path =  "metrics/" + str(ph*5) + "min/lasso/"    
    models = {}
    
    for key in data.keys():
        name = str(key) + "lasso" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"lasso",file_path,clarke_path,metric_path,ph)

In [ ]:
train_lasso_models(regression_dict_30,6)
train_lasso_models(regression_dict_60,12)

In [ ]:
predict_lasso_models(regression_dict_30,6)
predict_lasso_models(regression_dict_60,12)

# Elastic Net Regression

In [ ]:
def train_elasticnet_models(data,ph):
    """
    Train the Linear Regression Model with Elastic Net Regularization for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
    """
    path = "pickle/" + str(ph*5) + "min/elastic/"
    loop = tqdm(total = 6,position=0)
    for key in data.keys():
        lr = linear_model.ElasticNet(max_iter=10000)
        lr = MultiOutputRegressor(lr)
        lr.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "elasticnet" + str(ph*5)+ ".pickle"
        pickle.dump(lr,open(path+name,'wb'))
        loop.set_description("Done")
        loop.update(1)
    return

In [ ]:
def predict_elasticnet_models(data,ph):
    """
    Load each trained Linear Model with Elastic Net Regularization from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/elastic/"
    file_path = "plots/" + str(ph*5) + "min/elastic/"
    clarke_path = "clarke/" + str(ph*5) + "min/elastic/"
    metric_path =  "metrics/" + str(ph*5) + "min/elastic/"    
    models = {}
    for key in data.keys():
        name = str(key) + "elasticnet" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"elasticnet",file_path,clarke_path,metric_path,ph)

In [ ]:
train_elasticnet_models(regression_dict_30,6)
train_elasticnet_models(regression_dict_60,12)

In [ ]:
predict_elasticnet_models(regression_dict_30,6)
predict_elasticnet_models(regression_dict_60,12)

# KNN Regression

In [ ]:
def train_knn_models(data,ph):
    """
    Train the K Nearest Neighbors Regressor for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/knn/"
    for key in data.keys():
        knn = KNeighborsRegressor(n_jobs=-1)
        knn = MultiOutputRegressor(knn)
        knn.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "knn" + str(ph*5)+ ".pickle"
        pickle.dump(knn,open(path+name,'wb'))
    return

In [ ]:
def predict_knn_models(data,ph):
    """
    Load each K Nearest Neighbors Regressor from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/knn/"
    file_path = "plots/" + str(ph*5) + "min/knn/"
    clarke_path = "clarke/" + str(ph*5) + "min/knn/"
    metric_path =  "metrics/" + str(ph*5) + "min/knn/"    
    models = {}
    for key in data.keys():
        name = str(key) + "knn" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"knn",file_path,clarke_path,metric_path,ph)

In [ ]:
train_knn_models(regression_dict_30,6)
train_knn_models(regression_dict_60,12)

In [ ]:
predict_knn_models(regression_dict_30,6)
predict_knn_models(regression_dict_60,12)

# Random Forest Regression

In [ ]:
def train_rf_models(data,ph):
    """
    Train the Random Forest for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/rf/"
    loop = tqdm(total = 6,position=0)

    for key in data.keys():
        rf = RandomForestRegressor(max_depth = 4)
        rf = MultiOutputRegressor(rf)
        rf.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "rf" + str(ph*5)+ ".pickle"
        pickle.dump(rf,open(path+name,'wb'))
        loop.update(1)
    return

In [ ]:
def predict_rf_models(data,ph):
    """
    Load each Random Forest from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/rf/"
    file_path = "plots/" + str(ph*5) + "min/rf/"
    clarke_path = "clarke/" + str(ph*5) + "min/rf/"
    metric_path =  "metrics/" + str(ph*5) + "min/rf/"    
    models = {}
    for key in data.keys():
        name = str(key) + "rf" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"rf ",file_path,clarke_path,metric_path,ph)
    pass

In [ ]:
train_rf_models(regression_dict_30,6)
train_rf_models(regression_dict_60,12)

In [ ]:
predict_rf_models(regression_dict_30,6)

In [ ]:
predict_rf_models(regression_dict_60,12)

# Gradient Boosted Tree Regression

In [ ]:
def train_gradient_models(data,ph):
    """
    Train Gradient Boosted Trees for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/gradient/"
    loop = tqdm(total = 6,position=0)

    for key in data.keys():
        grad = GradientBoostingRegressor()
        grad = MultiOutputRegressor(grad)
        grad.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "gradient" + str(ph*5)+ ".pickle"
        pickle.dump(grad,open(path+name,'wb'))
        loop.update(1)
    return

In [ ]:
def predict_gradient_models(data,ph):
    """
    Load each trained Gradient Boosted Trees from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/gradient/"
    models = {}
    for key in data.keys():
        name = str(key) + "gradient" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    file_path = "plots/" + str(ph*5) + "min/gradient/"
    clarke_path = "clarke/" + str(ph*5) + "min/gradient/"
    metric_path =  "metrics/" + str(ph*5) + "min/gradient/"    
    predict_all(models,data,"gradient",file_path,clarke_path,metric_path,ph)
    pass

In [ ]:
train_gradient_models(regression_dict_30,6)

In [ ]:
predict_gradient_models(regression_dict_30,6)

In [ ]:
train_gradient_models(regression_dict_60,12)

In [ ]:
predict_gradient_models(regression_dict_60,12)

# XGBoost Regression

In [ ]:
def train_xgb_models(data,ph):
    """
    Train Extreme Gradient Boosted Trees Regressor for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(int): length of prediction horizon (take time frame and divide by 5) ie: 30min prediction = 6
    """
    path = "pickle/" + str(ph*5) + "min/xgb/"
    for key in data.keys():
        xgb = XGBRegressor(max_depth=4,n_jobs=-1,learning_rate=1)
        xgb = MultiOutputRegressor(xgb)
        param_grid = {"estimator__alpha":[0,0.5,1],"estimator__gamma":[0,0.5,1], "estimator__lambda":[1,2,3]}
        xgb_gs = GridSearchCV(xgb,param_grid,cv=5,refit=True,verbose=1,n_jobs=-1).fit(data[key]["train_data"],data[key]["train_labels"])
        best = xgb_gs.best_estimator_
        name = str(key) + "xgb" + str(ph*5)+ ".pickle"
        pickle.dump(best,open(path+name,'wb'))
    return

In [ ]:
def predict_xgb_models(data,ph):
    """
    Load each trained Extreme Gradient Boosted Trees from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/xgb/"
    models = {}
    for key in data.keys():
        name = str(key) + "xgb" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    file_path = "plots/" + str(ph*5) + "min/xgb/"
    clarke_path = "clarke/" + str(ph*5) + "min/xgb/"
    metric_path =  "metrics/" + str(ph*5) + "min/xgb/"    
    predict_all(models,data,"xgb",file_path,clarke_path,metric_path,ph)
    pass

In [ ]:
train_xgb_models(regression_dict_30,6)

In [ ]:
predict_xgb_models(regression_dict_30,6)

In [ ]:
train_xgb_models(regression_dict_60,12)

In [ ]:
predict_xgb_models(regression_dict_60,12)

# Multi Layer Perceptron

In [ ]:
def train_mlp_models(data,ph):
    """
    Train Extreme Gradient Boosted Trees Regressor for each person and saves them to pickle files
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(int): length of prediction horizon (take time frame and divide by 5) ie: 30min prediction = 6
    """
    path = "pickle/" + str(ph*5) + "min/mlp/"
    loop = tqdm(total = 6,position=0)
    ndim = 6*19
    for key in data.keys():
        mlp = MLPRegressor(hidden_layer_sizes=(100,100,100),solver="adam",max_iter=1000)
        mlp = MultiOutputRegressor(mlp)
        mlp.fit(data[key]["train_data"],data[key]["train_labels"])
        name = str(key) + "mlp" + str(ph*5)+ ".pickle"
        pickle.dump(mlp,open(path+name,'wb'))
        loop.update(1)
    return
    pass

In [ ]:
def predict_mlp_models(data,ph):
    """
    Load each MLPRegressor from the correct pickle file, then predict on test data for each model
    
    Parameters:
        data(dictionary): Dictionary containing the regression information for each patient
        ph(6 or 12): The length of the prediction horizon. 6 corresponds to 30-minutes and 12 corresponds to 60-minutes
    """
    path = "pickle/" + str(ph*5) + "min/mlp/"
    file_path = "plots/" + str(ph*5) + "min/mlp/"
    clarke_path = "clarke/" + str(ph*5) + "min/mlp/"
    metric_path =  "metrics/" + str(ph*5) + "min/mlp/"    
    models = {}
    for key in data.keys():
        name = str(key) + "mlp" + str(ph*5)+ ".pickle"
        models[key] = pickle.load(open(path+name,'rb'))
    predict_all(models,data,"mlp",file_path,clarke_path,metric_path,ph)
    pass

In [ ]:
train_mlp_models(regression_dict_30,6)

In [ ]:
predict_mlp_models(regression_dict_30,6)

In [ ]:
train_mlp_models(regression_dict_60,12)

In [ ]:
predict_mlp_models(regression_dict_60,12)

In [ ]:
#here we will include a comment about how a heavily engineered package like facebook's prophet can do

# Prophet

In [ ]:
df540 = parse_XML("OHIODATA/OhioT1DM-2-training/540-ws-training.xml")
temp = pd.DataFrame(np.array([df540["glucose_level"].index,df540["glucose_level"].values]).T, columns = ["ds","y"])
temp = temp.iloc[-18:-6]   #using the entire dataset just puts it around the average. This is the default linear method, it would be better to create a function for a different loss function as described in the documentation.

m = Prophet()
m = Prophet(changepoint_prior_scale=0.01).fit(temp)
future = m.make_future_dataframe(periods=1, freq='H')
fcst = m.predict(future)
fig = m.plot(fcst)

# Kalman Filters

https://stackoverflow.com/questions/43377626/how-to-use-kalman-filter-in-python-for-location-data

https://github.com/rlabbe/filterpy

## Kalman Filter

In [ ]:
#REGULAR KALMAN FILTER
def EKF(data, window = 6):
    my_filter = KalmanFilter(dim_x=2, dim_z=1)
    my_filter.x = np.array([[data[0]],[-0.9975063]])           # initial state (location and velocity)
    my_filter.F = np.array([[1.,1.],[0.,1.]])                  # state transition matrix
    my_filter.H = np.array([[1.,0.]])                          # Measurement function
    my_filter.P = np.array([[1000.,    0.], [   0., 1000.] ])  # covariance matrix
    my_filter.R = 5                                            # state uncertainty
    my_filter.Q = Q_discrete_white_noise(2, 0.1, .13)          # process uncertainty

    pred1 = []
    for i in range(1,len(data)):
        z = data[i]
        temp = my_filter
        for _ in range(window):         
            temp.predict()
        pred1.append(temp.x[0][0])
        my_filter.update(z)          #update with new data
    return pred1
    
dataukf = df540["glucose_level"].values
pred1 = EKF(dataukf, window = 12)


avgs = np.zeros(12)
for per in dfs_test:
    datakf = per["glucose_level"].values
    pred1 = EKF(datakf, window = 6)
    mets = get_metrics(datakf[206:], np.array(pred1[200:-5]))
    print()
    print("Person:")
    for i in range(len(mets)):
        avgs[i] += mets[i]
        print(labels[i] + " : " + str(mets[i]))
print(avgs/6)

## Unscented Kalman Filter

In [ ]:
#Unscented Kalman Filter
def UKF(window = 6):
    def fx(x, dt):
        # state transition function - predict next state based
        # on constant velocity model x = vt + x_0
        F = np.array([[48, 8, 0, 0],
                      [0, 48, 4, 0],
                      [0, 0, 48, 8],
                      [0, 0, 0, 48]], dtype=float)
        return np.dot(F, x)

    def hx(x):
       # measurement function - convert state into a measurement
       # where measurements are [x_pos, y_pos]
       return np.array([x[0], x[2]])

    dt = 12
    # create sigma points to use in the filter. This is standard for Gaussian processes
    points = MerweScaledSigmaPoints(4, alpha=.1, beta=2., kappa=-1)

    kfU = UnscentedKalmanFilter(dim_x=4, dim_z=2, dt=dt, fx=fx, hx=hx, points=points)
    kfU.x = np.array([measurements[0][0], measurements[0][1], 0., 0.]) # initial state
    kfU.P *= 0.1 # initial uncertainty
    z_std = 0
    kfU.R = np.diag([z_std**2, z_std**2]) # 1 standard
    kfU.Q = Q_discrete_white_noise(dim=2, dt=dt, var=0.01**2, block_size=2)

    predict = []
    for z in measurements:
        kfU.predict(dt = 6)
        predict.append(-1*kfU.x[2]/2270)
        kfU.update(z)
    return predict

dataukf = df540["glucose_level"].values
measurements = zip(range(len(dataukf)),dataukf)
measurements = np.array(tuple(measurements))
pred2 = UKF()
plot_compare(dataukf, pred2, 200, 600)

In [ ]:
avgs = np.zeros(12)
for per in dfs_test:
    dataukf = per["glucose_level"].values
    measurements = zip(range(len(dataukf)),dataukf)
    measurements = np.array(tuple(measurements))
    pred2 = UKF(window = 6)
    mets = get_metrics(dataukf[106:], np.array(pred2[100:-6]))
    print()
    print("Person:")
    for i in range(len(mets)):
        avgs[i] += mets[i]
        print(labels[i] + " : " + str(mets[i]))
print(avgs/6)

# ARMA (Autoregressive Moving Average)

In [ ]:
df540 = parse_XML("OHIODATA/OhioT1DM-2-training/540-ws-training.xml")
df544 = parse_XML("OHIODATA/OhioT1DM-2-training/544-ws-training.xml")
df552 = parse_XML("OHIODATA/OhioT1DM-2-training/552-ws-training.xml")
df567 = parse_XML("OHIODATA/OhioT1DM-2-training/567-ws-training.xml")
df584 = parse_XML("OHIODATA/OhioT1DM-2-training/584-ws-training.xml")
df596 = parse_XML("OHIODATA/OhioT1DM-2-training/596-ws-training.xml")

df540_test = parse_XML("OHIODATA/OhioT1DM-2-testing/540-ws-testing.xml")
df544_test = parse_XML("OHIODATA/OhioT1DM-2-testing/544-ws-testing.xml")
df552_test = parse_XML("OHIODATA/OhioT1DM-2-testing/552-ws-testing.xml")
df567_test = parse_XML("OHIODATA/OhioT1DM-2-testing/567-ws-testing.xml")
df584_test = parse_XML("OHIODATA/OhioT1DM-2-testing/584-ws-testing.xml")
df596_test = parse_XML("OHIODATA/OhioT1DM-2-testing/596-ws-testing.xml")

dfs_train = [df540,df544,df552,df567,df584,df596]
dfs_test = [df540_test,df544_test,df552_test,df567_test,df584_test,df596_test]

In [ ]:
#HALF HOUR PREDICTIONS
for i in range(6):
    pred_ARMA = []
    data_train = dfs_train[i]["glucose_level"].values
    data_test = dfs_test[i]["glucose_level"].values
    data = np.append(data_train,data_test)
    for i in range(len(data_test)-20):
        model = ARIMA(data[:len(data_train)+i], (2,0,2))        #set the middle value to 0 to get ARMA model
        model_fit = model.fit()
        pred_ARMA.append(model_fit.predict(len(data_train)+i, len(data_train)+i + 6)[-1])   
    mets = get_metrics(data[len(data_train)+6:len(data_train)+6+len(pred_ARMA)], np.array(pred_ARMA)[:])
    for i in range(len(mets)):
        print(labels[i] + " : " + str(mets[i]))
        
#ONE HOUR PREICTIONS
for i in range(len(dfs_test)):
    pred_ARMA = []
    data_train = dfs_train[i]["glucose_level"].values
    data_test = dfs_test[i]["glucose_level"].values
    data = np.append(data_train,data_test)
    for i in range(len(data_test)-20):
        model = ARIMA(data[:len(data_train)+i], (2,0,2))        #set the middle value to 0 to get ARMA model
        model_fit = model.fit()
        pred_ARMA.append(model_fit.predict(len(data_train)+i, len(data_train)+i + 12)[-1])   
    mets = get_metrics(data[len(data_train)+12:len(data_train)+12+len(pred_ARMA)], np.array(pred_ARMA)[:])
    for i in range(len(mets)):
        print(labels[i] + " : " + str(mets[i]))

In [ ]:
#Here is a way to find the p and q values using a grid search method
from pmdarima.arima import auto_arima
stepwise_model = auto_arima(y, start_p=2, start_q=1,
                           max_p=6, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

# VAR (Vectorized Autoregression)

In [ ]:
saved_pred2 = []
for k in range(len(dfs_train)):                                                                           #runs over each person 
    data = dfs_train[k].fillna(0)[["glucose_level", "basal", "bolus","exercise", "acceleration"]].values  #using lasso these are some of the significant columns in the predictions
    pred = []
    for i in range(len(data)-800):                                                                        #Using 800 previous made it stable enough to not throw errors
        model = VAR(endog=data[i:800+i])                                                                  #adds new data row each time we get it    
        model_fit = model.fit(maxlags=6)
        lag_order = model_fit.k_ar
        prediction = model_fit.forecast(data[i:800+i][-lag_order:], steps=12)
        #The predictions are in the form of an array, where each list represents the predictions of the row.
        #so the last row is the step number predictions for each dataframe cloumn value
        if prediction[-1][0] < 40 or prediction[-1][0] > 400:                                             #WORKING WITH PREDICTED VALUES OUTSIDE POSSIBLE RANGES
            prediction[-1][0] = prediction[-2][0]                                                                      #TODO: try prediction[-2][0]    #used 100 since that is near the average of the glucose values, should use the actual rollign average or a better method like the previous prediction
        pred.append(prediction[-1][0]) 
    saved_pred2.append(pred)

In [ ]:
#prints the metrics.
met_all = np.zeros(13)
for o in range(len(dfs_train)):
    mets = get_metrics(dfs_train[o]["glucose_level"].values[806:], np.array(saved_pred2[o][:-6]))
    for i in range(len(mets)):
        print(metric_labels[i] + " : " + str(mets[i]))
    met_all += mets
print(met_all/6)

# For the following ANFIS model, an older environment is needed. See link below for more information.

# ANFIS (Adaptive Neuro Fuzzy Inferance System)

https://github.com/tiagoCuervo/TensorANFIS

In [ ]:
class ANFIS:
    def __init__(self, n_inputs, n_rules, learning_rate=1e-2):
        self.n = n_inputs
        self.m = n_rules
        self.inputs = tf.placeholder(tf.float32, shape=(None, n_inputs))  # Input
        self.targets = tf.placeholder(tf.float32, shape=None)  # Desired output
        mu = tf.get_variable("mu", [n_rules * n_inputs],
                             initializer=tf.random_normal_initializer(0, 1))  # Means of Gaussian MFS
        sigma = tf.get_variable("sigma", [n_rules * n_inputs],
                                initializer=tf.random_normal_initializer(0, 1))  # Standard deviations of Gaussian MFS
        y = tf.get_variable("y", [1, n_rules], initializer=tf.random_normal_initializer(0, 1))  # Sequent centers

        self.params = tf.trainable_variables()

        self.rul = tf.reduce_prod(
            tf.reshape(tf.exp(-0.5 * tf.square(tf.subtract(tf.tile(self.inputs, (1, n_rules)), mu)) / tf.square(sigma)),
                       (-1, n_rules, n_inputs)), axis=2)  # Rule activations
        # Fuzzy base expansion function:
        num = tf.reduce_sum(tf.multiply(self.rul, y), axis=1)
        den = tf.clip_by_value(tf.reduce_sum(self.rul, axis=1), 1e-12, 1e12)
        self.out = tf.divide(num, den)

        #self.loss = tf.losses.huber_loss(self.targets, self.out)  # Loss function computation
        # Other loss functions for regression, uncomment to try them:
        self.loss = tf.sqrt(tf.losses.mean_squared_error(self.targets, self.out))
        # loss = tf.losses.absolute_difference(target, out)
        self.optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)  # Optimization step
        # Other optimizers, uncomment to try them:
        # self.optimize = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(self.loss)
        #self.optimize = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(self.loss)
        self.init_variables = tf.global_variables_initializer()  # Variable initializer

    def infer(self, sess, x, targets=None):
        if targets is None:
            return sess.run(self.out, feed_dict={self.inputs: x})
        else:
            return sess.run([self.out, self.loss], feed_dict={self.inputs: x, self.targets: targets})

    def train(self, sess, x, targets):
        yp, l, _ = sess.run([self.out, self.loss, self.optimize], feed_dict={self.inputs: x, self.targets: targets})
        return l, yp

In [ ]:
combined_all = []
comb_pred_all = []
mets_all = np.zeros(13)
for a in range(len(dfs_train)):
    combined_all.append(np.append(dfs_train[a]["glucose_level"].values,dfs_test[a]["glucose_level"].values))


#Gets predictions
for b in range(len(combined_all)):
    #make 12 to 6 for half hour, change 6 to 12 for hour
    tf.reset_default_graph()
    combined = combined_all[b] - 150
    
    D = 8  # number of regressors
    T = 1  # delay
    N = 14500  # Number of points to generate
    scale = 150
    num_epochs = 300

    combined_pred = np.array([])

    tf.reset_default_graph()
    mg_series = combined[-N:]/scale

    data = np.zeros((N - T - (D - 1) * T, D))  #creates the split data to train and test
    lbls = np.zeros((N - T - (D - 1) * T,))
    for t in range((D - 1) * T, N - 12):  
        data[t - (D - 1) * T, :] = [mg_series[t - 7 * T],mg_series[t - 6 * T],mg_series[t - 5 * T], mg_series[t - 4 * T],mg_series[t - 3 * T], mg_series[t - 2 * T], mg_series[t -1 * T], mg_series[t]]
        lbls[t - (D - 1) * T] = mg_series[t + 12]   
    trnData = data[:lbls.size - round(lbls.size * 0.3), :]
    trnLbls = lbls[:lbls.size - round(lbls.size * 0.3)]
    chkData = data[lbls.size - round(lbls.size * 0.3):, :]
    chkLbls = lbls[lbls.size - round(lbls.size * 0.3):]


    # ANFIS params and Tensorflow graph initialization
    m = 20  # number of rules     usually 16
    alpha = 0.01  # learning rate
    fis = ANFIS(n_inputs=D, n_rules=m, learning_rate=alpha)

    # Initialize session to make computations on the Tensorflow graph
    with tf.Session() as sess:
        # Initialize model parameters
        sess.run(fis.init_variables)
        trn_costs = []
        val_costs = []
        for epoch in range(num_epochs):
            #  Run an update step
            trn_loss, trn_pred = fis.train(sess, trnData, trnLbls)
            # Evaluate on validation set
            val_pred, val_loss = fis.infer(sess, chkData, chkLbls)
            if epoch == num_epochs - 1:
                pred = np.vstack((np.expand_dims(trn_pred, 1), np.expand_dims(val_pred, 1)))
            trn_costs.append(trn_loss)
            val_costs.append(val_loss)
        combined_pred = np.append(combined_pred,np.expand_dims(val_pred, 1))   

    #In get metrics function comment out the squared argument on RMSE score
    mets = get_metrics(combined[-len(combined_pred)+12:][:-200]+150, np.array(combined_pred[:-12]*scale+150)[:-200])
    for i in range(len(mets)):
        print(metric_labels[i] + " : " + str(mets[i]))
    comb_pred_all.append(combined_pred)
    mets_all +=mets
print(mets_all/6)
    

# While the following models weren't included in the final results, some small work was done for these models. The code is left here as such

# CNN (Convolutional Neural Net)

https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

In [ ]:
df_cols = ["date","glucose_level", "finger_stick", "basal", "temp_basal","bolus","meal","sleep",
           "work","stressors","hypo_event","illness","exercise","basis_heart_rate","basis_gsr",
          "basis_skin_temperature","basis_air_temperature","basis_steps","basis_sleep","acceleration"]
df_cnn_train = dfs_train[0].fillna(0)
df_cnn_test = dfs_test[0].fillna(0)

In [ ]:
# multivariate multi-step 1d cnn example
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def create_input(df_vals,start,size):
    x_input = []
    for i in range(size):
        x_input.append(df_vals[start+i])
    return np.array(x_input)

# define input sequence
# convert to [rows, columns] structure
N = len(df_cnn_train["glucose_level"].values)

in_seq1 = np.array(df_cnn_train["glucose_level"].values[:N-6]).reshape((N-6, 1))
in_seq2 = np.array(df_cnn_train["finger_stick"].values[:N-6]).reshape((N-6, 1))
in_seq3 = np.array(df_cnn_train["basal"].values[:N-6]).reshape((N-6, 1))
in_seq4 = np.array(df_cnn_train["temp_basal"].values[:N-6]).reshape((N-6, 1))
in_seq5 = np.array(df_cnn_train["bolus"].values[:N-6]).reshape((N-6, 1))
in_seq6 = np.array(df_cnn_train["meal"].values[:N-6]).reshape((N-6, 1))
in_seq7 = np.array(df_cnn_train["sleep"].values[:N-6]).reshape((N-6, 1))
in_seq8 = np.array(df_cnn_train["stressors"].values[:N-6]).reshape((N-6, 1))
in_seq9 = np.array(df_cnn_train["hypo_event"].values[:N-6]).reshape((N-6, 1))
in_seq10 = np.array(df_cnn_train["illness"].values[:N-6]).reshape((N-6, 1))
in_seq11 = np.array(df_cnn_train["exercise"].values[:N-6]).reshape((N-6, 1))
in_seq12 = np.array(df_cnn_train["basis_heart_rate"].values[:N-6]).reshape((N-6, 1))
in_seq13 = np.array(df_cnn_train["basis_gsr"].values[:N-6]).reshape((N-6, 1))
in_seq14 = np.array(df_cnn_train["basis_skin_temperature"].values[:N-6]).reshape((N-6, 1))
in_seq15 = np.array(df_cnn_train["basis_air_temperature"].values[:N-6]).reshape((N-6, 1))
in_seq16 = np.array(df_cnn_train["basis steps"].values[:N-6]).reshape((N-6, 1))          
in_seq17 = np.array(df_cnn_train["basis_sleep"].values[:N-6]).reshape((N-6, 1))
in_seq18 = np.array(df_cnn_train["acceleration"].values[:N-6]).reshape((N-6, 1))
in_seq19 = np.array(df_cnn_train["work"].values[:N-6]).reshape((N-6, 1))

out_seq = np.array(df_cnn_train["glucose_level"].values[6:N]).reshape((N-6, 1))

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq19, in_seq8, in_seq9, in_seq10, in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, in_seq17, in_seq18, out_seq))

# choose a number of time steps
n_steps_in, n_steps_out = 30, 6                      #TODO steps in can be hypertuned
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model

model.fit(X, y, epochs=500, verbose=2)

In [ ]:
#Gets the predictions from the fitted model
pred = []
for i in range(800):
    x_input = create_input(df_cnn_test.values,440+i,30)  #[[df_rnn.values[445+i][1:]]
    x_input = x_input.reshape((1, n_steps_in, n_features))
    pred.append(model.predict(x_input, verbose=0)[0,-1])
    
mets = get_metrics(df_cnn_test["glucose_level"].values[482:1282], np.array(pred))
for i in range(len(mets)):
    print(labels[i] + " : " + str(mets[i]))

In [ ]:
#Here is a second attempt with more layers
n_steps_in, n_steps_out = 30, 6                      #TODO steps in can be hypertuned
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(Conv1D(filters=400, kernel_size=12, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(350, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model

model.fit(X, y, epochs=2000, verbose=2)

pred = []
for i in range(800):
    x_input = create_input(df_rnn.values,440+i,30)  #[[df_rnn.values[445+i][1:]]
    x_input = x_input.reshape((1, n_steps_in, n_features))
    pred.append(model.predict(x_input, verbose=0)[0,-1])

# LSTM

https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [ ]:
# univariate cnn lstm example
# choose a number of time steps
n_steps_in, n_steps_out = 24, 6
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)

pred = []
for i in range(800):
    x_input = create_input(df_rnn.values,440+i,24)  #[[df_rnn.values[445+i][1:]]
    x_input = x_input.reshape((1, n_steps_in, n_features))
    pred.append(model.predict(x_input, verbose=0)[0,-1])

# CNN-LSTM

https://machinelearningmastery.com/cnn-long-short-term-memory-networks/

In [ ]:
# choose a number of time steps
n_steps_in, n_steps_out = 36, 6
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2

n_features = X.shape[2]

n_features = 19                      #(2**3 , 3 is leftovers)   (2**2, 41, 73 is len X), (19 is features)
n_seq = 6
n_steps = 6
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

# define model
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu'), input_shape=(6, n_steps, n_features)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(50, activation='relu'))

model.add(Flatten())

model2.add(Dense(6))
model2.compile(optimizer='adam', loss='mse')
# fit model
model2.fit(X, y, epochs=3000, verbose=2)

pred = []
for i in range(800):
    x_input = create_input(df_rnn.values,440+i,36) 
    x_input = x_input.reshape((1,n_seq, n_steps, n_features))
    pred.append(model2.predict(x_input, verbose=0)[0,-1])